# Softmax Cost Function 구현하기

Softmax는 Logistic Regression과 비슷한 Logic으로 진행된다.

Logistic Regression이 binary classification이라면, Softmax는 multipe classification이다.

Softmax에 관한 내용은 제 노션에 정리 되어 있습니다.

Softmax에 대한 정리: https://noversezero.notion.site/Softmax-Regression-443d6bf93b234f11a3e6134130953d7b

Cross Entropy를 구하기 위한 MLE 정리: https://noversezero.notion.site/Maximum-likelihood-estimation-20d7543aac854715a611d29c30b2c18e

## Low-Level

In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


3가지 원소를 가진 벡터 텐서를 정의하고, 이를 softmax의 입력 값으로 사용하여 결과를 확인해보았습니다.

결과값을 보면, 3개의 원소의 값이 0과 1사이라는 것을 알 수 있습니다.

이 값들은 각 원소별 확률입니다.

따라서 이 원소들의 합은 1이될 것입니다.

이를 확인해보겠습니다.

In [3]:
hypothesis.sum()

tensor(1.)

확인해본 결과 1이 맞았습니다.

이제 Cost Function을 직접 구현해보겠습니다.

In [5]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward0>)


먼저 3 x 5 행렬의 크기를 갖는 텐서를 만들어보았습니다.

이 행렬에 대해서 Softmax 함수를 적용하면, 각 값에 따라 확률값을 예측합니다. *dim = 1은 두번째 차원에 대해서 Softmax를 적용한다는 의미입니다. 

즉, 3개의 Sample에 대해서 5개의 Class 중에 정답 확률을 예측한 결과입니다.

따라서 각 행의 원소들의 함이 1이되는 텐서가 되었습니다.

In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([3, 1, 2])


먼저 각 Sample에 대해서 임의로 레이블을 만들겠습니다.

이는 각 Sample의 정답값이 될 것입니다.

이를 이용해 각 레이블에 대해서 One-hot-Encoding을 하겠습니다.

In [7]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

위 코드의 2번째 식을 해석해보겠습니다.

scatter의 첫번째 인자로 dim=1에 대해서 수행하라고 알려주고, 세번째 인자에 숫자 1을 넣어주므로서 두번째 인자인 y_unsqeeze(1)이 알려주는 위치에 숫자 1을 넣도록 합니다.

즉, 두번째 차원에 대해 y_unsqeeze(1)이 알려주는 위치에 1을 넣으라는 의미입니다.

y_unsqeeze(1)를 하면 (3,)의 크기를 가졌던 y 텐서는 (3 × 1) 텐서가 됩니다. 

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4992, grad_fn=<MeanBackward0>)


이제 비용함수 연산을 하면된다.

Softmax Cost Function은 위에서 구한 hypothesis에 log를 씌운 값을 One-hot-vecotr에 곱한 값들의 합을 평균낸 것이다.

## High Level

위에서는 Low Level로 Softmax Cost Function을 구현했는데, 이제는 High Level로 구현해보겠습니다.

먼저 Low Level에서는 Softmax 값을 출력한 다음에 해당 값에 torch.log()를 사용했습니다.

하지만, PyTorch에서는 두 함수를 결합한 F.log_softmax()라는 함수가 있습니다.

In [10]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward0>)

In [9]:
# Softmax의 출력값에 Log를 바로 씌워서 출력하는 함수
F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward0>)

이번에는 앞서 Cost Function을 직접 수식화해서 코딩한 것과 다르게 코딩이 가능합니다.

위에서 Softmax의 출력값에 Log를 씌우는 함수를 배웠습니다.

이를 통해 Cost Function을 정의하는데 더 편해졌지만, 더 간단하게 F.null_loss()를 사용하면 One-Hot_Vector를 사용할 필요없이 바로 Cost 값을 출력합니다.

이때, null은 Negative Log Likelihood의 약자로 MLE에 대한 내용은 위의 Notion을 참조하시면 됩니다.

더 간단하게도 F.cross_entropy()라는 함수를 사용하면 바로 Cost 값을 출력할 수 있습니다. 

In [11]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4992, grad_fn=<MeanBackward0>)

In [12]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4992, grad_fn=<NllLossBackward0>)

In [13]:
F.cross_entropy(z, y)

tensor(1.4992, grad_fn=<NllLossBackward0>)